In [4]:
#encoding:utf8
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from datetime import datetime
from WindPy import w

w.start()

#获取各个资产的收益率数据
def get_revenue_data(indict,start_date,end_date):
    wsd_data = w.wsd(indict,"pct_chg",start_date,end_date,"PriceAdj=F")
    df = pd.DataFrame(np.array(wsd_data.Data).T,index=wsd_data.Times,columns=wsd_data.Codes)
    df = df.fillna(0)
    df.index = pd.to_datetime(df.index)
    return df.sort_index()


#获取交易日期序列,频率： {周：W；月：M；季：Q；年：Y}
def get_td_series(start_date,end_date,period):
    w_data = w.tdays(start_date,end_date,"Period="+period)    
    return [x.strftime('%Y-%m-%d') for x in w_data.Data[0]]

#获取偏移日期
def get_offset_date(n,t_date,period=""):
    w_data = w.tdaysoffset(n, t_date, "Period="+period)
    return w_data.Data[0][0].strftime('%Y-%m-%d')

#计算资产组合的风险值
def calc_risk_portfolio(weight,cov_matrix):
    weight = np.matrix(weight)
    sigma = np.sqrt(weight*cov_matrix*weight.T)
    MRC = cov_matrix*weight.T/sigma
    RC = np.multiply(MRC,weight.T)
    return RC

#定义优化问题的目标函数
def risk_objective(x_weight,args):
    x_weight = np.matrix(x_weight)
    cov_matrix = args[0]
    RC_target_ratio = args[1]
    sigma_portfolio = np.sqrt(x_weight*cov_matrix*x_weight.T)
    
    RC_target = np.asmatrix(np.multiply(sigma_portfolio,RC_target_ratio))
    RC_real   = calc_risk_portfolio(x_weight,cov_matrix)
    sum_squared_diff = sum(np.square(RC_real-RC_target.T))[0][0]
    return sum_squared_diff

#优化问题的第一个约束条件
def cons_1(x_weight):
    return np.sum(x_weight)-1.0

#优化问题的第二个约束条件
def cons_2(x_weight):
    return x_weight

#根据资产预期目标风险贡献度来计算各资产的权重
def calc_weight_portfolio(RC_target_ratio,weight_default,cov_matrix):
    cons = ({'type':'eq','fun':cons_1},{'type':'ineq',"fun":cons_2})
    res = minimize(risk_objective,weight_default,args=[cov_matrix,RC_target_ratio],method='SLSQP',constraints=cons,options={'disp':True})
    weight_final = np.asmatrix(res.x)
    return weight_final


start_date = '2005-01-01'
end_date = '2018-12-28'

#沪深300、中证500、恒生指数、标普500、日经225、ICE布油连续、COMEX黄金连续
fields = ['000300.SH','HSI.HI','SPX.GI','B00.IPE','GC00.CMX']
indict = ','.join(fields)

df = get_revenue_data(indict,start_date,end_date)
df.to_excel("asset_pctchg.xlsx",sheet_name="data")
df

,000300.SH,HSI.HI,SPX.GI,B00.IPE,GC00.CMX
2005-01-04,-1.720600,-1.345188,-1.167144,1.433515,-0.116360
2005-01-05,0.994105,-2.004428,-0.362779,-1.291423,-0.442684
2005-01-06,-0.946035,-0.380112,0.350584,5.776352,-1.333957
2005-01-07,0.079742,-1.000435,-0.143111,0.770128,-0.498102
2005-01-10,1.008275,-0.320224,0.342272,-0.602131,0.047676
2005-01-11,0.327605,-0.163620,-0.609956,0.465983,0.643317
2005-01-12,-0.038811,0.414975,0.398144,1.298701,0.994318
2005-01-13,0.012942,0.058753,-0.863013,3.502747,-0.351617
2005-01-14,-0.859785,-0.578342,0.600450,-0.132714,-0.494001
2005-01-17,-2.110075,0.940141,0.000000,-0.265781,0.000000
